In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

def generate_dataset(num_examples):
    """Generate a dataset of Tic-Tac-Toe boards and labels"""
    X = np.zeros((num_examples, 9))
    y = np.zeros((num_examples, 9))
    z = np.zeros((num_examples))
    for i in range(num_examples):
        # Generate a random Tic-Tac-Toe board
        board = np.zeros((3, 3))
        player = 1
        move_count = 0
        while True:
            # Make a valid move
            empty_cells = np.argwhere(board == 0)
            if len(empty_cells) == 0:
                break
            idx = np.random.randint(len(empty_cells))
            row, col = empty_cells[idx]
            board[row][col] = player
            move_count += 1
            # Check for a win or a draw
            if move_count >= 5:
                if (np.any(np.sum(board, axis=0) == 3) or np.any(np.sum(board, axis=1) == 3) or 
                    np.sum(np.diag(board)) == 3 or np.sum(np.diag(np.fliplr(board))) == 3):
                    # player 1 wins
                    break
                elif (np.any(np.sum(board, axis=0) == -3) or np.any(np.sum(board, axis=1) == -3) or 
                      np.sum(np.diag(board)) == -3 or np.sum(np.diag(np.fliplr(board))) == -3):
                    # player 2 wins
                    break
                elif np.all(board != 0):
                    # game is a draw
                    break
            # Switch players after each move
            player = 1 if player == 2 else 2
        X[i] = board.flatten()
        # Calculate the label for the board
        if (np.any(np.sum(board, axis=0) == 3) or np.any(np.sum(board, axis=1) == 3) or 
        np.sum(np.diag(board)) == 3 or np.sum(np.diag(np.fliplr(board))) == 3):
        # player 1 wins
          y[i] = np.array([1 if cell == 1 else 0 for cell in board.flatten()])
          z[i] = 1
        elif (np.any(np.sum(board, axis=0) == -3) or np.any(np.sum(board, axis=1) == -3) or 
              np.sum(np.diag(board)) == -3 or np.sum(np.diag(np.fliplr(board))) == -3):
            # player 2 wins
            y[i] = np.array([1 if cell == 2 else 0 for cell in board.flatten()])
            z[i] = -1
        else:
            # game is a draw
            y[i] = np.zeros((9))
            z[i] = 0
    return X, y, z

def print_board(board):
  """Print the Tic-Tac-Toe board"""
  print("  0 1 2")
  for i in range(3):
    print(i, end=" ")
    row = " ".join(["X" if cell == 1 else "O" if cell == 2 else " " for cell in board[i]])
    print(row)
   
def get_move(board, player, model):
    """Get the next move for the specified player using the model"""
    if player == 1:
        # Human player's turn
        try:
            row = int(input("Enter row index: "))
            col = int(input("Enter column index: "))
            if row >= 0 and row < 3 and col >= 0 and col < 3 and board[row][col] == 0:
                return row, col
            else:
                print("Invalid move, try again.")
        except ValueError:
            print("Invalid input, try again.")
    else:
        # AI player's turn
        while True:
            # Use the model to predict the best move
            predictions = model.predict(np.array([board.flatten()]))
            move = np.argmax(predictions)
            row = move // 3
            col = move % 3
            if board[row][col] == 0:
                return row, col
            # Add a break statement to exit the loop if a valid move is not found
            break
    # Return a default move if the human player entered invalid input or if the AI player is stuck in an infinite loop
    return 0, 0

def play_game(model):
    """Play a game of Tic-Tac-Toe against the AI"""
    board = np.zeros((3, 3))
    player = 1
    while True:
        print_board(board)
        if player == 1:
            # Human player's turn
            row, col = get_move(board, player, model)
        else:
            # AI's turn
            row, col = get_move(board, player, model)
        board[row][col] = player
        if (np.all(board[0] == player) or np.all(board[1] == player) or np.all(board[2] == player) or 
            np.all(board[:,0] == player) or np.all(board[:,1] == player) or np.all(board[:,2] == player) or 
            np.all(np.diag(board) == player) or np.all(np.diag(np.fliplr(board)) == player)):
            # player wins
            print(f"Player {player} wins!")
            break
        elif np.all(board != 0):
            # game is a draw
            print("The game is a draw.")
            break
        player = 1 if player == 2 else 2

def train(num_examples, model):
    # Generate a dataset
    X, y = generate_dataset(num_examples)

    # Train the model
    model.fit(X, y, epochs=10, verbose=1)

def train(X, y, model, epochs=10, batch_size=32):
    """Train the model on the Tic-Tac-Toe dataset"""
    model.compile(loss='categorical_crossentropy', optimizer=Adam())
    model.fit(X, y, epochs=epochs, batch_size=batch_size)


# Generate a dataset of 10000 Tic-Tac-Toe boards
X, y, z = generate_dataset(10000)

# Define the model
model = Sequential()
model.add(Dense(9, input_dim=9, activation='relu'))
model.add(Dense(9, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the generated data
history = model.fit(X, y, batch_size=32, epochs=10, verbose=1)

# Play a game of Tic-Tac-Toe against the AI
play_game(model)

Epoch 1/10
313/313 [==============================] - 1s 3ms/step - loss: 0.6690 - accuracy: 0.1111
Epoch 2/10
313/313 [==============================] - 1s 3ms/step - loss: 0.6305 - accuracy: 0.0993
Epoch 3/10
313/313 [==============================] - 1s 3ms/step - loss: 0.6117 - accuracy: 0.1160
Epoch 4/10
313/313 [==============================] - 1s 3ms/step - loss: 0.5748 - accuracy: 0.0806
Epoch 5/10
313/313 [==============================] - 1s 3ms/step - loss: 0.5268 - accuracy: 0.0752
Epoch 6/10
313/313 [==============================] - 1s 3ms/step - loss: 0.4933 - accuracy: 0.1218
Epoch 7/10
313/313 [==============================] - 1s 3ms/step - loss: 0.4715 - accuracy: 0.1444
Epoch 8/10
313/313 [==============================] - 1s 2ms/step - loss: 0.4548 - accuracy: 0.1533
Epoch 9/10
313/313 [==============================] - 1s 3ms/step - loss: 0.4393 - accuracy: 0.1590
Epoch 10/10
313/313 [==============================] - 1s 3ms/step - loss: 0.4214 - accuracy: 0.1743